In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 9.3 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 15.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.6 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.8 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 9.3 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 31.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 50.0 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 30.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 25.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━

In [2]:
from datasets import load_dataset

# Assuming you've already loaded your dataset
dataset = load_dataset("huggingartists/snoop-dogg")

# Split the dataset into training and validation sets
train_test_split = dataset["train"].train_test_split(test_size=0.1)

Found cached dataset snoop-dogg (/home/mehr/.cache/huggingface/datasets/huggingartists___snoop-dogg/default/1.0.0/aa170dcc8ff7afd4f239ce2eb60aacfe1c41a29f1f553500cdcc661beccdf26a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

def tokenize_function(examples):
    # Tokenize the texts and map the tokens to their IDs
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Apply tokenization to the dataset
tokenized_datasets = train_test_split.map(tokenize_function, batched=True)

# Make sure the labels are correctly set for language modeling
tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['input_ids']}, batched=True)

Map:   0%|          | 0/1595 [00:00<?, ? examples/s]

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1595 [00:00<?, ? examples/s]

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

In [6]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    num_train_epochs=5,                # Number of training epochs
    per_device_train_batch_size=4,     # Batch size per device during training
    per_device_eval_batch_size=4,      # Batch size for evaluation
    evaluation_strategy="epoch",       # Evaluate at the end of each epoch
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],  # Note: 'test' here is the validation set we created
)

trainer.train()

/home/mehr/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.402247
2,3.578100,3.345195
3,3.171600,3.312766
4,3.027100,3.323739
5,3.027100,3.327674


TrainOutput(global_step=1995, training_loss=3.1603652074522244, metrics={'train_runtime': 415.2699, 'train_samples_per_second': 19.204, 'train_steps_per_second': 4.804, 'total_flos': 520950988800000.0, 'train_loss': 3.1603652074522244, 'epoch': 5.0})

In [7]:
trainer.save_model("snoop-dogg-model")

In [32]:
from transformers import pipeline

generator = pipeline('text-generation', model="snoop-dogg-model", tokenizer='gpt2')

# prompt = "Just chillin', you know how we do, "
prompt = "how do you wake up my child"
texts = generator(prompt, max_length=100, num_return_sequences=1)

generated_text = ""
for text in texts:
    print(text['generated_text'])
    generated_text += text['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


how do you wake up my child mama?
You know I got the keys so I could cash
I have a little something for your mothers
I guess Im going to have to wait for you
So my little something for you daddy
Honey, I never been to the beach
Its a tradition, but I aint never been to the beach
So I can give you something just so you know it
So come and get it
Just call my daddy
Treat me


In [34]:
import requests
import json
import os

text_to_speak = "This is the text you want to convert to speech."

response = requests.request(
  method="POST",
  url="https://api.neets.ai/v1/tts",
  headers={
    "Content-Type": "application/json",
    "X-API-Key": "c0d9aeeef6f14e98ad46c8d23e24535b"
  },
  json={
    "text": generated_text,
    "voice_id": "snoop-dogg",
    "params": {
      "model": "ar-diff-50k"
    }
  }
)

with open("neets_demo.mp3", "wb") as f:
  f.write(response.content)

audio_file = "neets_demo.mp3"
os.system("mpg321 " + audio_file)

High Performance MPEG 1.0/2.0/2.5 Audio Player for Layer 1, 2, and 3.
Version 0.3.2-1 (2012/03/25). Written and copyrights by Joe Drew,
now maintained by Nanakos Chrysostomos and others.
Uses code from various people. See 'README' for more!
THIS SOFTWARE COMES WITH ABSOLUTELY NO WARRANTY! USE AT YOUR OWN RISK!
tcgetattr(): Inappropriate ioctl for device

Playing MPEG stream from neets_demo.mp3 ...
MPEG 2.0 layer III, 128 kbit/s, 24000 Hz mono
                                                                            
[0:20] Decoding of neets_demo.mp3 finished.


0

In [1]:
# Check GPU working
import torch

print(torch.version.cuda)
if torch.cuda.is_available():
    print("CUDA (GPU) is available and enabled!")
    device = torch.device("cuda")
else:
    print("CUDA (GPU) is not available. Training will run on CPU.")
    device = torch.device("cpu")

CUDA (GPU) is available and enabled!
